In [1]:
import os
import tweepy as tw
import pandas as pd
import re
import nltk
import numpy as np
from wordcloud import WordCloud,STOPWORDS
nltk.download('punkt')   
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from textblob import TextBlob
from datetime import datetime,timedelta ,date
# from pythainlp.corpus import thai_stopwords
# from pythainlp.tokenize import word_tokenize
import requests
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Teaw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Teaw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
consumer_key= 'gOoM90Ig2rs8POa8wxzPgw0oD'
consumer_secret= '74vY9D6ZmoMkO4zFFv0uZL9Sl3u2Y7UzibGZqtK1j5bN6agxU5'
access_token= '1503736251530170368-rC1gNRmBIgs7HGdgSYuokoUYRKLij6'
access_token_secret= '9I8ndE2w6jPVpod5n0YNNS7732VU4stmL8MAjHvcso5xT'

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
class ClawTwitter:
    def __init__(self, keyw:str):
        self.listdata = []
        self.keyword = keyw
        self.data = self.get_datatweet()

    def connect(self):
        try:
            auth = tw.OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_token_secret)
            api = tw.API(auth)
            return api
        except:
            print("Error")
            exit(1)

    def remove_url(txt):
        """Replace URLs found in a text string with nothing 
        (i.e. it will remove the URL from the string).

        Parameters
        ----------
        txt : string
            A text string that you want to parse and remove urls.

        Returns
        ----------
        The same txt string with url's removed.
        """

        return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

    def remove_url_th(txt):
        """Replace URLs found in a text string with nothing 
        (i.e. it will remove the URL from the string).

        Parameters
        ----------
        txt : string
            A text string that you want to parse and remove urls.

        Returns
        ----------
        The same txt string with url's removed.
        """

        return " ".join(re.sub("([^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''|(\w+:\/\/\S+))", "", txt).split())

    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r'(@[A-Za-z0-9_]+)', '', text)
        text = re.sub('http://\S+|https://\S+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text_tokens = word_tokenize(text)
        text = [word for word in text_tokens if not word in stopwords.words()]
        text = ' '.join(text)
        return text


    def clean_text_th(self, text):
        text = re.sub('http://\S+|https://\S+', '', text)
        url = "https://api.aiforthai.in.th/textcleansing"
        params = {f'text':{text}}
        headers = {
            'Apikey': "DXApQFOxFoYXJWd2i0QoCRis4YJfp7az",
            }
        res = requests.request("GET", url, headers=headers, params=params)
        # print(res.json())
        return res.json()['cleansing_text']


    def stemmer(self, text):
        porter_text = PorterStemmer()
        token_words = word_tokenize(text)
        stem_word = []
        for word in token_words:
            stem_word.append(porter_text.stem(word))
        return " ".join(stem_word)


    def sentiment(self, text_en):
        sentiment_text = text_en.sentiment.polarity
        if  sentiment_text > 0:
            return 'positive'
        elif sentiment_text < 0:
            return 'negative'
        else:
            return 'neutral'


    def sentiment_th(self, text_th):
        url = "https://api.aiforthai.in.th/ssense"
        params = {'text':text_th}
        headers = {
            'Apikey': "DXApQFOxFoYXJWd2i0QoCRis4YJfp7az"
            }
        res = requests.get(url, headers=headers, params=params)
        if res.json()['sentiment']['polarity'] == '':
            polarity = "neutral"
        else:
            polarity = res.json()['sentiment']['polarity']
        # print('hey', res.json())
        return polarity


    def get_datatweet(self):
        api = self.connect()
        today = datetime.utcnow().date()
        today_str = today.strftime("%Y-%m-%d")
        tweets = tw.Cursor(api.search_tweets,
                    q=self.keyword + " -filter:retweets",
                    since= today_str).items(100)
                    # since= today_str).items(10)
                     
        tweet_list = []    
        for tweet in tweets:
            tweet_list.append(tweet)
        tweets = tweet_list

        data_tweet = [[self.keyword,
                        tweet.user.screen_name,
                        # tweet.lang,
                        tweet.user.location if tweet.user.location != '' else 'unknown',
                        tweet.created_at.replace(tzinfo = None) + timedelta(hours = 7),
                        ClawTwitter.remove_url(tweet.text) if tweet.lang == "en" else ClawTwitter.remove_url_th(tweet.text),
                        tweet.retweet_count,
                        tweet.favorite_count,
                        # self.sentiment(TextBlob(self.stemmer(self.clean_text(ClawTwitter.remove_url(tweet.text))))),
                        # self.sentiment(TextBlob(self.stemmer(ClawTwitter.remove_url(tweet.text)))),
                        self.sentiment(TextBlob(self.stemmer(self.clean_text(ClawTwitter.remove_url(tweet.text)))))
                                        if tweet.lang == "en" else self.sentiment_th(self.clean_text_th(tweet.text)),
                        # self.sentiment_th(tweet.text) if tweet.lang == "th"
                        #             else self.sentiment(TextBlob(self.stemmer(self.clean_text(ClawTwitter.remove_url(tweet.text))))),
                        f"https://twitter.com/twitter/statuses/{tweet.id}"] for tweet in tweets]

        return data_tweet


    def show_data(self, data_tweet:list):
        tweet_frame = pd.DataFrame(data=data_tweet,
                            # columns=['topic', 'user','language','location','post date', 'tweet','retweet','likes', 'sentiment','tweet link'])
                            columns=['topic', 'user','location','post date', 'tweet','retweet','likes', 'sentiment','tweet link'])
        
        # tweet_frame.to_excel(f"{self.keyword}.xlsx", engine="openpyxl", index=False)
        return tweet_frame


    def twitter_to_xlsx(self, tweets):
        hashtag = self.keyword
        today = date.today()
        today_str = today.strftime("%d%m%Y")
        tweets.to_excel(hashtag+"_"+today_str+".xlsx", engine="openpyxl", index=False)
    

    def data_hashtagcount(self):
        api = self.connect()
        today = datetime.utcnow().date()
        today_str = today.strftime("%Y-%m-%d")
        new_column = []
        for tweet in tw.Cursor(api.search_tweets,
                    q=self.keyword + " -filter:retweets",
                    since= today_str).items(100):
            entity_ht = tweet.entities.get('hashtags')
            hashtag = ""
            for i in range(0, len(entity_ht)):
                hashtag = hashtag + ",#" + entity_ht[i]['text']
            new_column.append([hashtag])
        data_frame = pd.DataFrame(data = new_column, columns=["hashtag"])
        return data_frame
    

    def twitterhashtage_to_xlsx(self, datframe):
        hashtag = self.keyword
        today = date.today()
        today_str = today.strftime("%d%m%Y")
        datframe.to_excel(hashtag+"_"+"hashcount"+"_"+today_str+".xlsx", engine="openpyxl", index=False)


    def slash_tokenize(self,data):  
        result = data.split(",")
        return result


    def count_wordframe(self):
        topic = self.keyword
        today = date.today()
        today_str = today.strftime("%d%m%Y")
        readfile = pd.read_excel(topic+"_"+"hashcount"+"_"+today_str+".xlsx")
        hastag_data = readfile["hashtag"].dropna()
        vectorizer = CountVectorizer(tokenizer = self.slash_tokenize)
        transformed_data = vectorizer.fit_transform(hastag_data)
        hash_tag_cnt_df = pd.DataFrame(columns = ['word', 'count']) 
        hash_tag_cnt_df['word'] = vectorizer.get_feature_names()
        hash_tag_cnt_df['count'] = np.ravel(transformed_data.sum(axis=0))
        tophash_tag = hash_tag_cnt_df.sort_values(by=['count'], ascending=False).head(20)
        return tophash_tag

    def twittercount_to_xlsx(self, datframe):
        hashtag = self.keyword
        today = date.today()
        today_str = today.strftime("%d%m%Y")
        datframe.to_excel(hashtag+"_"+"count"+"_"+today_str+".xlsx", engine="openpyxl", index=False)



    def tokenize_en(self, d):
        # print('hey', d)
        d = re.sub('[ก-๙]', '', d)
        d = re.sub('[0-9]', '', d)
        # print('test d', d)
        result = d.split(" ")
        result = list(filter(None, result))
        return result

    def count_word_en(self, dfr):
        new_text = []
        for txt in dfr["tweet"]:
            new_text.append(self.clean_text(txt))
        
        vectorizer = CountVectorizer(tokenizer=self.tokenize_en)
        transformed_data = vectorizer.fit_transform(new_text)
        keyword_df_en = pd.DataFrame(columns = ['word', 'count'])
        keyword_df_en['word'] = vectorizer.get_feature_names()
        keyword_df_en['count'] = np.ravel(transformed_data.sum(axis=0))
        topword_en = keyword_df_en.sort_values(by=['count'], ascending=False).head(20)

        return topword_en


    # def tokenize_th(self, d):
    #     # print('hey', d)
    #     d = re.sub('[A-Za-z]', '', d)
    #     d = re.sub('[0-9]', '', d)
    #     # print('test d', d)
    #     result = d.split(" ")
    #     # print('re', result)
    #     result = list(filter(None, result))
    #     return result

    # def test_th(self, data):
    #     # a = self.tokenize_th(data)
    #     url ='https://api.aiforthai.in.th/ssense'
    #     headers = {'Apikey':'DXApQFOxFoYXJWd2i0QoCRis4YJfp7az'}
    #     params = {'text': data }
    #     # res = requests.get(url, headers=headers, params=params)
    #     # # print(res.status_code)
    #     # # print(res.headers)
    #     # print(res.json())
    #     res = requests.request("GET", url, headers=headers, params=params)
    #     # print(res.json())
    #     return res.json()['tokens']

    # def count_word_th(self, dfr):
    #     new_text = []
    #     for txt in dfr["tweet"]:
    #         new_text.append(self.clean_text_th(txt))
    #     # print('test new', new_text)
        
    #     vectorizer = CountVectorizer(tokenizer=self.tokenize_th)
    #     transformed_data = vectorizer.fit_transform(new_text)
    #     # count_data = zip(vectorizer.get_feature_names(), np.ravel(transformed_data.sum(axis=0)))
    #     keyword_df_th = pd.DataFrame(columns = ['word', 'count'])
    #     keyword_df_th['word'] = vectorizer.get_feature_names()
    #     keyword_df_th['count'] = np.ravel(transformed_data.sum(axis=0))
    #     topword_th = keyword_df_th.sort_values(by=['count'], ascending=False).head(20)

    #     return topword_th


    # def sentiment_th_test(self, text_th):
    #     url = "https://api.aiforthai.in.th/ssense"
    #     params = {'text':text_th}
    #     headers = {
    #         'Apikey': "DXApQFOxFoYXJWd2i0QoCRis4YJfp7az"
    #         }
    #     res = requests.get(url, headers=headers, params=params)
    #     # if res.json()['sentiment']['polarity'] == '':
    #     #     polarity = "neutral"
    #     # else:
    #     #     polarity = res.json()['sentiment']['polarity']
    #     # print('hey', res.json())
    #     # a = res.json()['preprocess']['segmented']
    #     print('hey', res.json()['preprocess']['segmented'])
    #     return res.json()['preprocess']['segmented']


    # def tokenize(self, d):
    #     result = d.split("/")
    #     result = list(filter(None, result))
    #     return result

    # def test_w(self, tweet_dataf):
    #     new_text = []
    #     for txt in tweet_dataf["tweet"]:
    #         new_text.append(self.sentiment_th_test(txt))
    #     if txt.lang == "th":
    #         vectorizer = CountVectorizer(tokenizer=self.tokenize)
    #         transformed_data = vectorizer.fit_transform(new_text)
    #         count_data = zip(vectorizer.get_feature_names(), np.ravel(transformed_data.sum(axis=0)))
    #         keyword_df2 = pd.DataFrame(columns = ['word', 'count'])
    #         keyword_df2['word'] = vectorizer.get_feature_names()
    #         keyword_df2['count'] = np.ravel(transformed_data.sum(axis=0))
    #         keyword_df2.sort_values(by=['count'], ascending=False).head(10)
    #     return keyword_df2


In [5]:
# test = ClawTwitter("#REDVELVET", "th")
test = ClawTwitter("#REDVELVET")

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


JSONDecodeError: [Errno Expecting value] : 0

In [ ]:
test.show_data(test.data)

: 

In [ ]:
tweets_data = test.show_data(test.data)
test.twitter_to_xlsx(tweets_data)

: 

In [ ]:
tweets_data = test.show_data(test.data)
test.count_word_en(tweets_data)

: 

In [ ]:
# tweets_data = test.show_data(test.data)
# test.count_word_th(tweets_data)

: 

In [ ]:
test.data_hashtagcount()

: 

In [ ]:
hashtag_data = test.data_hashtagcount()
test.twitterhashtage_to_xlsx(hashtag_data)

: 

In [ ]:
test.count_wordframe()

: 

In [ ]:
count_wordf = test.count_wordframe()
test.twittercount_to_xlsx(count_wordf)

: 